## Imports

In [18]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten

In [19]:
df = pd.read_csv('SoFla Neighborhoods New Construction - DT_FTL_useme.csv')

In [20]:
df.columns

Index(['address', 'stories', 'units', 'dt_fl_developer', 'trd_developer',
       'developer_for_map', 'description', 'status', 'story link',
       'useful_info'],
      dtype='object')

## Image URL collector

In [21]:
### Drop last n rows of a df. In case there are sums that need clipped
## df.drop(df.tail(n).index, inplace=True)

In [22]:
df['image_links'] = ''
links = []
for url in df['story link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find_all('img')
        counter = 0
        for image in images:
            counter += 1
            if 'jpg' in image['src']:
                links.append(image['src'])
                if counter == len(images):
                    links.append('no image found')
                break
            else:
                links.appedn('no image found')
    except:
        links.append('no_image')

df['image_links'] = links

## Article link formatter

In [23]:
df.description

0                    ECI Flagler Village LLC, 270 units
1     Downtown Fort Lauderdale Waterfront 18 LLC, 65...
2                               509 NE 3, LLC, 26 units
3                              200 third LLC, 388 units
4                             418 NE2 Ave LLC, 53 units
5     Apartment giants Kushner Companies and Aimco p...
6     Traina Companies, a co-developer of Privé at I...
7                                FLAT 404 LLC, 44 units
8      BLF 2021 LLC, Station Village Florida, 400 units
9                         Barron Real Estate, 320 units
10    Apartment giants Kushner Companies and Aimco p...
11    Brooklyn-based Dependable Equities plans a 1.5...
12                      Third Avenue Dev LLC, 512 units
13                                  SWFA LLC. 380 units
14                     New River Central LLC, 407 units
15    Ricardo Vadia, who leads Miami-based ROVR deve...
16    Traina Companies, a co-developer of Privé at I...
17                                 SWFLB LLC, 36

In [25]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story link"] + '">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0     <a href="na">ECI Flagler Village</a> LLC, 270 ...
1     <a href="na">Downtown Fort</a> Lauderdale Wate...
2              <a href="na">509 NE 3, LLC, 26</a> units
3             <a href="na">200 third LLC, 388</a> units
4            <a href="na">418 NE2 Ave LLC, 53</a> units
5     <a href="https://therealdeal.com/miami/2022/01...
6     <a href="https://therealdeal.com/miami/2019/01...
7               <a href="na">FLAT 404 LLC, 44</a> units
8     <a href="na">BLF 2021 LLC,</a> Station Village...
9        <a href="na">Barron Real Estate,</a> 320 units
10    <a href="https://therealdeal.com/miami/2022/01...
11    <a href="https://therealdeal.com/miami/2022/06...
12     <a href="na">Third Avenue Dev</a> LLC, 512 units
13                 <a href="na">SWFA LLC. 380 units</a>
14    <a href="na">New River Central</a> LLC, 407 units
15    <a href="https://therealdeal.com/miami/2018/02...
16    <a href="https://therealdeal.com/miami/2019/01...
17                <a href="na">SWFLB LLC, 362 un

In [ ]:
df.story_link[0]

## Google Maps API Geolocater Setup

In [26]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [28]:
df['geo_address'] = df['address'] + ', Fort Lauderdale, FL'
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## HTML popup formatter

In [32]:
def popup_html(row):
    i = row
    address = df['address'].iloc[i]
    developer = df['developer_for_map'].iloc[i]
    description = df['description_link'].iloc[i]
    image = df['image_links'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>
    <strong>{}'''.format(description) + '''</strong></html>
    '''
    return html


## Map Maker

In [ ]:
df.description_link

In [34]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=13)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'New Development in Downtown Fort Lauderdale')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [ ]:
m.save('index.html')